In [1]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
import polars as pl
from sqlalchemy import create_engine,text
from scipy import stats
import plotly.express as px
import toml
import geopandas as gpd
import plotly.express as px
import plotly.graph_objects as go
import h5py

%matplotlib inline
from IPython.display import display, HTML

relative_path = "../../../.."

config = toml.load(Path.cwd() / '../../../../configuration/input_configuration.toml')
summary_config = toml.load(Path.cwd() / '../../../../configuration/summary_configuration.toml')

pd.options.display.float_format = '{:0,.0f}'.format

# Transit Boardings

In [2]:

df = pd.read_csv(os.path.join(relative_path,'outputs','transit','daily_boardings_by_agency.csv'),index_col=0)
df.loc['Region Total','boardings'] = df['boardings'].sum()
df = df.reset_index()
df.rename(columns={'agency_name': 'Agency', 'boardings': 'Boardings'}, inplace=True)
HTML(df.to_html(index=False))

Agency,Boardings
King County Metro,"792,203"
Sound Transit,"521,005"
Community Transit,"110,397"
Pierce Transit,"73,664"
Kitsap Transit,"28,766"
Everett Transit,"21,466"
Washington Ferries,"16,148"
Region Total,"1,563,650"


In [3]:
df = pd.read_csv(os.path.join(relative_path,'outputs','transit','transit_line_results.csv'),index_col=0)

transit_type_map = {
    1: 'Local Bus',
    2: 'Express Bus',
    3: 'Bus Rapid Transit',
    4: 'Streetcar',
    5: 'Commuter Rail',
    6: 'Light Rail',
    7: 'Auto Ferry',
    8: 'Passenger Ferry'
}
df['transit_type'] = df['transit_type'].map(transit_type_map)
df = df.groupby('transit_type').sum()[['boardings']]
df.loc['Total'] = df.sum(axis=0)
df = df.reset_index()
df.rename(columns={'boardings':' Boardings','transit_type':'Transit Type'}, inplace=True)
HTML(df.to_html(index=False))

Transit Type,Boardings
Auto Ferry,"19,367"
Bus Rapid Transit,"404,538"
Commuter Rail,"9,048"
Light Rail,"478,098"
Local Bus,"617,389"
Passenger Ferry,"21,657"
Streetcar,"13,552"
Total,"1,563,650"


In [4]:
df = pd.read_csv(os.path.join(relative_path,'outputs','transit','transit_line_results.csv'),index_col=0)
df = df[(df['agency_code']==6)].groupby('transit_type').sum()[['boardings']]
df.index = df.index.map({1: 'Bus', 3: 'BRT (Stride)', 5: 'Commuter Rail', 6: 'Light Rail'})
df

,boardings
transit_type,
Bus,"10,591"
BRT (Stride),"23,267"
Commuter Rail,"9,048"
Light Rail,"478,098"


In [17]:
# Light rail boardings by station
df_boardings = pd.read_csv(os.path.join(relative_path,'outputs','transit','boardings_by_stop.csv'))
df_transit_stops = pd.read_csv(os.path.join(relative_path,'inputs','scenario','networks','transit_stops.csv'))

# df_transit_stops = summary_data.load_agg_data('../inputs/scenario/networks/transit_stops.csv')
light_rail_df = df_transit_stops[df_transit_stops['light_rail']==1]
light_rail_df
light_rail_df = light_rail_df[['PSRCJunctID', 'x', 'y']]

df = pd.merge(df_boardings, light_rail_df, left_on=['i_node'], right_on=['PSRCJunctID'])

# # # Load station names
conn = create_engine('sqlite:///../../../../inputs/db/'+config['db_name'])
df_sql = pd.read_sql(
        "SELECT station_name, emme_node FROM light_rail_station_boardings",
        con=conn,
    )
# df_sql = summary_data.load_sqlite()
# df_sql
# # # Merge station names to df 
df = df.merge(df_sql, left_on=['PSRCJunctID'], right_on=['emme_node'])
# # df.to_clipboard()
# # df_sql
df = df[['total_boardings','station_name','PSRCJunctID']].drop_duplicates()

# # pivot table of boardings by station 
# df = df.pivot_table(index='station_name', columns='source', values='total_boardings', aggfunc='sum')
# df.style.format('{:,.0f}', na_rep='-')
df[['PSRCJunctID','station_name','total_boardings']]

,PSRCJunctID,station_name,total_boardings
0,197649,Husky Stadium,"11,864"
5,198007,Westlake,"32,464"
10,198009,Symphony,"17,966"
15,198011,Pioneer Square,"19,642"
20,198013,Int'l District,"27,939"
...,...,...,...
172,198129,Avalon Station,"2,549"
173,198130,West Seattle Junction Station,"5,917"
174,198352,Boeing Access Station,76
175,198888,Graham Station,"1,817"


In [5]:
df = pd.read_csv(os.path.join(relative_path,'outputs','transit','total_transit_trips.csv'),index_col=0)
df.rename(index={'commuter_rail': 'Commuter Rail',
                'litrat': 'Light Rail',
                'ferry': 'Auto Ferry',
                'passenger_ferry': 'Passenger Ferry',
                'trnst': 'Bus'}, inplace=True)
df.columns = ['Total Trips']
df.loc['Total'] = df.sum(axis=0)
df.reset_index(inplace=True)
df.rename(columns={'index': 'Transit Type'}, inplace=True)
HTML(df.to_html(index=False))

Transit Type,Total Trips
Commuter Rail,"9,790"
Light Rail,"442,116"
Auto Ferry,"26,550"
Passenger Ferry,"11,209"
Bus,"576,644"
Total,"1,066,309"


# Households Near HCT

In [6]:
# List of Stations
# Load transit stops file

#"The definition we have been using for the RTP is BRT, LRT, Commuter Rail, StreetCar and Ferry."
df = pd.read_csv(r'../../../../inputs/scenario/networks/transit_stops.csv')
# Streetcar is coded as light rail
df['hct'] = 0
df.loc[df[['commuter_rail','light_rail','ferry','brt']].sum(axis=1) > 0,'hct'] = 1
df_hct = df[df['hct'] == 1]

# Map of Stations
# Load as a geodataframe
gdf_hct = gpd.GeoDataFrame(
    df_hct, geometry=gpd.points_from_xy(df_hct.x, df_hct.y))

gdf_hct.crs = 'EPSG:2285'

In [7]:
df_lu = pl.read_csv(r'..\..\..\..\inputs\scenario\landuse\parcels_urbansim.txt', separator=" ").to_pandas()

# Load as a geodataframe
gdf_lu = gpd.GeoDataFrame(
    df_lu, geometry=gpd.points_from_xy(df_lu.xcoord_p, df_lu.ycoord_p))

gdf_lu.crs = 'EPSG:2285'

In [8]:
parcel_geog = pd.read_sql_table('parcel_'+config['base_year']+'_geography', 'sqlite:///../../../../inputs/db/'+config['db_name'])

In [9]:
df_lu = df_lu.merge(parcel_geog, left_on='parcelid', right_on='ParcelID', how='left')
# Add a field that defines whether a parcel is inside (1) or outside (0) any RGC
df_lu['RGC_binary'] = 0
df_lu.loc[df_lu['GrowthCenterName'] != 'Not in RGC', 'RGC_binary'] = 1

In [10]:
def calculate_buffer(gdf_lu, _gdf_hct, distance):
    
    # Buffer the HCT station gdf
    _gdf_hct['geometry'] = _gdf_hct.buffer(distance)

    gdf_intersect = gpd.overlay(_gdf_hct, gdf_lu, how="intersection", keep_geom_type=False)
    df = df_lu[df_lu['parcelid'].isin(gdf_intersect['parcelid'].unique())]
    
    return df, gdf_intersect

def aggregate_parcels(df, col_dict, sum_field):
    results_df = pd.DataFrame()
    for col, name in col_dict.items():
        _df = df[[col,sum_field]].groupby(col).sum()[[sum_field]]
        _df['group'] = name
        results_df = pd.concat([results_df,_df])
    results_df = results_df.reset_index()
    
    return results_df

In [11]:
df_025, gdf_025 = calculate_buffer(gdf_lu, gdf_hct.copy(), distance=5280.0/4)

In [12]:
df_050, gdf_050 = calculate_buffer(gdf_lu, gdf_hct.copy(), distance=5280.0/2)

In [13]:
pd.options.display.float_format = '{:0,.0f}'.format
col_dict = {'equity_focus_areas_2023__efa_poc': 'People of Color',
                      'equity_focus_areas_2023__efa_pov200': 'Poverty',
                        'equity_focus_areas_2023__efa_lep': 'LEP',
                      'equity_focus_areas_2023__efa_older': 'Older',
                      'equity_focus_areas_2023__efa_youth': 'Youth',
            'equity_focus_areas_2023__efa_dis': 'Disability',
           'rg_proposed': 'Regional Geography',
            'CountyName': 'County',
            'GrowthCenterName': 'Regional Growth Center',
            'RGC_binary': 'RGC Binary',
            'Region': 'Region'
           }

hct_hh_df = pd.DataFrame()
df_025 = df_025.copy()
df_025['Region'] = 1
df = aggregate_parcels(df_025, col_dict, 'hh_p')
df['distance'] = 0.25
hct_hh_df = pd.concat([hct_hh_df,df])
df_050 = df_050.copy()
df_050['Region'] = 1
df = aggregate_parcels(df_050, col_dict, 'hh_p')
df['distance'] = 0.50
hct_hh_df = pd.concat([hct_hh_df,df])

In [14]:
pd.options.display.float_format = '{:0,.0f}'.format

In [15]:

df = hct_hh_df[hct_hh_df['group'] == 'Region']
df = df.rename(columns={'hh_p': 'Total'}).copy()
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'Region'
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df = df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'})

df['% of Households (1/4 Mile)'] = df['1/4 Mile']/df_lu['hh_p'].sum()
df['% of Households (1/2 Mile)'] = df['1/2 Mile']/df_lu['hh_p'].sum()

output = df.to_html(formatters={
    '% of Households (1/4 Mile)': '{:0,.1%}'.format,
    '% of Households (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

Region,1/4 Mile,1/2 Mile,% of Households (1/4 Mile),% of Households (1/2 Mile)
1,"186,999","363,813",10.8%,21.0%


In [16]:
pd.options.display.float_format = '{:0,.0f}'.format
df = hct_hh_df[hct_hh_df['group'] == 'County'].copy()
df = df.rename(columns={'hh_p': 'Total'}).copy()
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'County'
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)

_df = df.merge(df_lu[['CountyName','hh_p']].groupby('CountyName').sum()[['hh_p']].reset_index(),left_on='County', right_on='CountyName')
_df['% of Households (1/4 Mile)'] = _df['1/4 Mile']/_df['hh_p']
_df['% of Households (1/2 Mile)'] = _df['1/2 Mile']/_df['hh_p']

df = df.merge(_df[['County', '% of Households (1/4 Mile)', '% of Households (1/2 Mile)']], 
              on='County')

output = df.to_html(formatters={
    '% of Households (1/4 Mile)': '{:0,.1%}'.format,
    '% of Households (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

County,1/4 Mile,1/2 Mile,% of Households (1/4 Mile),% of Households (1/2 Mile)
King,"164,432","299,497",17.2%,31.3%
Kitsap,203,"1,600",0.2%,1.5%
Pierce,"3,106","9,595",0.9%,2.7%
Snohomish,"19,258","53,121",6.0%,16.6%


In [17]:
pd.options.display.float_format = '{:0,.0f}'.format
df = hct_hh_df[hct_hh_df['group'] == 'RGC Binary'].copy()
df.rename(columns={'hh_p': 'Total'}, inplace=True)
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df = df.fillna(0)
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'RGC Binary'
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)


_df = df.merge(df_lu[['RGC_binary','hh_p']].groupby('RGC_binary').sum()[['hh_p']].reset_index(),left_on='RGC Binary', 
               right_on='RGC_binary')
_df['% of Households (1/4 Mile)'] = _df['1/4 Mile']/_df['hh_p']
_df['% of Households (1/2 Mile)'] = _df['1/2 Mile']/_df['hh_p']

df = df.merge(_df[['RGC Binary', '% of Households (1/4 Mile)', '% of Households (1/2 Mile)']], 
              on='RGC Binary')
df['RGC Designation'] = df['RGC Binary'].map({0: 'Outside RGC', 1: 'Inside RGC'})
df = df[['RGC Designation','1/4 Mile','1/2 Mile','% of Households (1/4 Mile)', '% of Households (1/2 Mile)']]
output = df.to_html(formatters={
    '% of Households (1/4 Mile)': '{:0,.1%}'.format,
    '% of Households (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

RGC Designation,1/4 Mile,1/2 Mile,% of Households (1/4 Mile),% of Households (1/2 Mile)
Outside RGC,"112,210","234,652",7.1%,15.0%
Inside RGC,"74,789","129,161",44.9%,77.5%


In [18]:
pd.options.display.float_format = '{:0,.0f}'.format
df = hct_hh_df[hct_hh_df['group'] == 'Regional Growth Center'].copy()
df.rename(columns={'hh_p': 'Total'}, inplace=True)
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df = df.fillna(0)
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'Regional Growth Center'
# Ensure all RGCs are included, even those that have no HCT access
for center in parcel_geog['GrowthCenterName'].unique():
    if center not in df.index:
        df.loc[center] = 0

df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)

# Calcualte the percent of households in this geography within 1/4 and 1/2 mile
_df = df.merge(df_lu[['GrowthCenterName','hh_p']].groupby('GrowthCenterName').sum()[['hh_p']].reset_index(),left_on='Regional Growth Center', 
               right_on='GrowthCenterName')
_df['% of Households (1/4 Mile)'] = _df['1/4 Mile']/_df['hh_p']
_df['% of Households (1/2 Mile)'] = _df['1/2 Mile']/_df['hh_p']

df = df.merge(_df[['Regional Growth Center', '% of Households (1/4 Mile)', '% of Households (1/2 Mile)']], 
              on='Regional Growth Center')

df = df.sort_values('Regional Growth Center')

output = df.to_html(formatters={
    '% of Households (1/4 Mile)': '{:0,.1%}'.format,
    '% of Households (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

Regional Growth Center,1/4 Mile,1/2 Mile,% of Households (1/4 Mile),% of Households (1/2 Mile)
Auburn,719,982,71.6%,97.8%
Bellevue,"3,035","8,844",30.9%,90.1%
Bothell Canyon Park,0,210,0.0%,73.4%
Bremerton,121,611,7.3%,37.0%
Burien,"1,789","1,967",91.0%,100.0%
Everett,"1,444","3,175",37.7%,83.0%
Federal Way,234,234,100.0%,100.0%
Greater Downtown Kirkland,0,0,0.0%,0.0%
Issaquah,0,0,NaN,NaN
Kent,318,962,32.9%,99.5%


In [19]:
pd.options.display.float_format = '{:0,.0f}'.format
df = hct_hh_df[hct_hh_df['group'] == 'Regional Geography'].copy()
df.rename(columns={'hh_p': 'Total'}, inplace=True)
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df = df.fillna(0)
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'Regional Geography'
df.reset_index(inplace=True)

df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)
HTML(df.to_html(index=False))

_df = df.merge(df_lu[['rg_proposed','hh_p']].groupby('rg_proposed').sum()[['hh_p']].reset_index(),left_on='Regional Geography', 
               right_on='rg_proposed')
_df['% of Households (1/4 Mile)'] = _df['1/4 Mile']/_df['hh_p']
_df['% of Households (1/2 Mile)'] = _df['1/2 Mile']/_df['hh_p']

df = df.merge(_df[['Regional Geography', '% of Households (1/4 Mile)', '% of Households (1/2 Mile)']], 
              on='Regional Geography')

output = df.to_html(formatters={
    '% of Households (1/4 Mile)': '{:0,.1%}'.format,
    '% of Households (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

Regional Geography,1/4 Mile,1/2 Mile,% of Households (1/4 Mile),% of Households (1/2 Mile)
Cities and Towns,0,195,0.0%,0.1%
Core Cities,"32,160","63,828",8.3%,16.5%
High Capacity Transit Communities,"21,390","46,432",5.9%,12.9%
Metropolitan Cities,"132,699","250,425",22.5%,42.5%
Rural Areas,116,554,0.1%,0.3%
Urban Unincorporated Areas,634,"2,379",1.0%,3.8%


In [20]:
pd.options.display.float_format = '{:0,.2f}'.format
# df = hct_hh_df.copy()
# df = df[df['index'] == 1]
# df.rename(columns={'hh_p': 'Total'}, inplace=True)
# df.drop('index', axis=1, inplace=True)
# df = df.pivot_table(index='group', columns='distance', values='Total', aggfunc='sum')
# df[0.25] = df[0.25].astype('float32')
# df[0.5] = df[0.5].astype('float32')
# df.index.name = 'Equity Geography'
# df = df.reset_index()
# df = df.rename_axis(None, axis=1)
# df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)

In [21]:
pd.options.display.float_format = '{:0,.1f}'.format

equity_dict = {'equity_focus_areas_2023__efa_poc': 'People of Color',
                      'equity_focus_areas_2023__efa_pov200': 'Poverty',
                        'equity_focus_areas_2023__efa_lep': 'LEP',
                      'equity_focus_areas_2023__efa_older': 'Older',
                      'equity_focus_areas_2023__efa_youth': 'Youth',
            'equity_focus_areas_2023__efa_dis': 'Disability'}

df = hct_hh_df[hct_hh_df['group'].isin(equity_dict.values())].copy()
df['EFA Type'] = df['index'].map({
                                0: 'Below Regional Average', 
                                1: 'Above Regional Average', 
                                2: 'Higher Share of Equity Population',
                                })

df = df.pivot_table(index=['group','EFA Type'], columns='distance', values='hh_p', aggfunc='sum').reset_index()

# Merge with total households to get shares of group
efa_pop_tot = df_lu[equity_dict.keys()].sum().to_dict()

for lu_col, label in equity_dict.items():
    df.loc[df['group'] == label, "Total Pop"] = efa_pop_tot[lu_col]
df['% of Households (1/4 Mile)'] = df[0.25]/df['Total Pop']
df['% of Households (1/2 Mile)'] = df[0.5]/df['Total Pop']

df.rename(columns={'group': 'Group', 0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)
df = df.rename_axis(None, axis=1)
df.drop('Total Pop', inplace=True, axis=1)
output = df.to_html(formatters={
    '% of Households (1/4 Mile)': '{:0,.1%}'.format,
    '% of Households (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

Group,EFA Type,1/4 Mile,1/2 Mile,% of Households (1/4 Mile),% of Households (1/2 Mile)
Disability,Above Regional Average,55283,107001,7.0%,13.5%
Disability,Below Regional Average,102238,200637,12.9%,25.4%
Disability,Higher Share of Equity Population,29478,56175,3.7%,7.1%
LEP,Above Regional Average,44669,80560,7.7%,14.0%
LEP,Below Regional Average,94508,186165,16.4%,32.3%
LEP,Higher Share of Equity Population,47822,97088,8.3%,16.8%
Older,Above Regional Average,39325,81037,4.2%,8.7%
Older,Below Regional Average,129071,248310,13.8%,26.6%
Older,Higher Share of Equity Population,18603,34466,2.0%,3.7%
People of Color,Above Regional Average,73690,135087,11.6%,21.3%


# Jobs Near HCT

In [22]:
pd.options.display.float_format = '{:0,.0f}'.format
col_dict = {'equity_focus_areas_2023__efa_poc': 'People of Color',
                      'equity_focus_areas_2023__efa_pov200': 'Poverty',
                        'equity_focus_areas_2023__efa_lep': 'LEP',
                      'equity_focus_areas_2023__efa_older': 'Older',
                      'equity_focus_areas_2023__efa_youth': 'Youth',
            'equity_focus_areas_2023__efa_dis': 'Disability',
           'rg_proposed': 'Regional Geography',
            'CountyName': 'County',
            'GrowthCenterName': 'Regional Growth Center',
            'RGC_binary': 'RGC Binary',
            'Region': 'Region'
           }

hct_hh_df = pd.DataFrame()
df_025['Region'] = 1
df = aggregate_parcels(df_025, col_dict, 'emptot_p')
df['distance'] = 0.25
hct_hh_df = pd.concat([hct_hh_df,df])
df_050['Region'] = 1
df = aggregate_parcels(df_050, col_dict, 'emptot_p')
df['distance'] = 0.50
hct_hh_df = pd.concat([hct_hh_df,df])

In [23]:

df = hct_hh_df[hct_hh_df['group'] == 'Region'].copy()
df.rename(columns={'emptot_p': 'Total'}, inplace=True)
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'Region'
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)

df['% of Jobs (1/4 Mile)'] = df['1/4 Mile']/df_lu['emptot_p'].sum()
df['% of Jobs (1/2 Mile)'] = df['1/2 Mile']/df_lu['emptot_p'].sum()

output = df.to_html(formatters={
    '% of Jobs (1/4 Mile)': '{:0,.1%}'.format,
    '% of Jobs (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

Region,1/4 Mile,1/2 Mile,% of Jobs (1/4 Mile),% of Jobs (1/2 Mile)
1,"603,448","913,238",28.0%,42.4%


In [24]:
pd.options.display.float_format = '{:0,.0f}'.format
df = hct_hh_df[hct_hh_df['group'] == 'County'].copy()
df.rename(columns={'emptot_p': 'Total'}, inplace=True)
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'County'
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)

_df = df.merge(df_lu[['CountyName','emptot_p']].groupby('CountyName').sum()[['emptot_p']].reset_index(),left_on='County', right_on='CountyName')
_df['% of Jobs (1/4 Mile)'] = _df['1/4 Mile']/_df['emptot_p']
_df['% of Jobs (1/2 Mile)'] = _df['1/2 Mile']/_df['emptot_p']

df = df.merge(_df[['County', '% of Jobs (1/4 Mile)', '% of Jobs (1/2 Mile)']], 
              on='County')

output = df.to_html(formatters={
    '% of Jobs (1/4 Mile)': '{:0,.1%}'.format,
    '% of Jobs (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

County,1/4 Mile,1/2 Mile,% of Jobs (1/4 Mile),% of Jobs (1/2 Mile)
King,"540,420","772,584",37.5%,53.6%
Kitsap,"1,040","12,395",1.1%,13.6%
Pierce,"19,094","36,528",5.7%,10.9%
Snohomish,"42,894","91,731",15.0%,32.1%


In [25]:
pd.options.display.float_format = '{:0,.0f}'.format
df = hct_hh_df[hct_hh_df['group'] == 'RGC Binary'].copy()
df.rename(columns={'emptot_p': 'Total'}, inplace=True)
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df = df.fillna(0)
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'RGC Binary'
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)


_df = df.merge(df_lu[['RGC_binary','emptot_p']].groupby('RGC_binary').sum()[['emptot_p']].reset_index(),left_on='RGC Binary', 
               right_on='RGC_binary')
_df['% of Jobs (1/4 Mile)'] = _df['1/4 Mile']/_df['emptot_p']
_df['% of Jobs (1/2 Mile)'] = _df['1/2 Mile']/_df['emptot_p']

df = df.merge(_df[['RGC Binary', '% of Jobs (1/4 Mile)', '% of Jobs (1/2 Mile)']], 
              on='RGC Binary')
df['RGC Designation'] = df['RGC Binary'].map({0: 'Outside RGC', 1: 'Inside RGC'})
df = df[['RGC Designation','1/4 Mile','1/2 Mile','% of Jobs (1/4 Mile)', '% of Jobs (1/2 Mile)']]
output = df.to_html(formatters={
    '% of Jobs (1/4 Mile)': '{:0,.1%}'.format,
    '% of Jobs (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

RGC Designation,1/4 Mile,1/2 Mile,% of Jobs (1/4 Mile),% of Jobs (1/2 Mile)
Outside RGC,"162,837","324,577",11.6%,23.1%
Inside RGC,"440,611","588,661",58.7%,78.4%


In [26]:
pd.options.display.float_format = '{:0,.0f}'.format
df = hct_hh_df[hct_hh_df['group'] == 'Regional Growth Center'].copy()
df.rename(columns={'emptot_p': 'Total'}, inplace=True)
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df = df.fillna(0)
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'Regional Growth Center'
# Ensure all RGCs are included, even those that have no HCT access
for center in parcel_geog['GrowthCenterName'].unique():
    if center not in df.index:
        df.loc[center] = 0
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)
df['% of Total Jobs (1/4 Mile)'] = df['1/4 Mile']/df_lu['emptot_p'].sum()
df['% of Total Jobs (1/2 Mile)'] = df['1/2 Mile']/df_lu['emptot_p'].sum()

output = df.to_html(formatters={
    '% of Total Jobs (1/4 Mile)': '{:0,.1%}'.format,
    '% of Total Jobs (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

Regional Growth Center,1/4 Mile,1/2 Mile,% of Total Jobs (1/4 Mile),% of Total Jobs (1/2 Mile)
Auburn,"2,331","3,845",0.1%,0.2%
Bellevue,"33,006","51,248",1.5%,2.4%
Bothell Canyon Park,"4,236","7,776",0.2%,0.4%
Bremerton,657,"10,940",0.0%,0.5%
Burien,"3,578","3,758",0.2%,0.2%
Everett,"8,878","13,550",0.4%,0.6%
Federal Way,"1,195","2,358",0.1%,0.1%
Kent,"3,076","6,053",0.1%,0.3%
Not in RGC,"162,837","324,577",7.6%,15.1%
Puyallup Downtown,"1,815","3,112",0.1%,0.1%


In [27]:
df = hct_hh_df[hct_hh_df['group'] == 'Regional Geography'].copy()
df.rename(columns={'emptot_p': 'Total'}, inplace=True)
df = df.pivot_table(index='index', columns='distance', values='Total', aggfunc='sum')
df = df.fillna(0)
df[0.25] = df[0.25].astype('float32')
df[0.5] = df[0.5].astype('float32')
df.index.name = 'Regional Geography'
df = df.reset_index()
df = df.rename_axis(None, axis=1)
df.rename(columns={0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)
df['% of Total Jobs (1/4 Mile)'] = df['1/4 Mile']/df_lu['emptot_p'].sum()
df['% of Total Jobs (1/2 Mile)'] = df['1/2 Mile']/df_lu['emptot_p'].sum()

output = df.to_html(formatters={
    '% of Total Jobs (1/4 Mile)': '{:0,.1%}'.format,
    '% of Total Jobs (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

Regional Geography,1/4 Mile,1/2 Mile,% of Total Jobs (1/4 Mile),% of Total Jobs (1/2 Mile)
Cities and Towns,0,109,0.0%,0.0%
Core Cities,"153,548","226,413",7.1%,10.5%
High Capacity Transit Communities,"25,667","45,912",1.2%,2.1%
Metropolitan Cities,"422,344","635,954",19.6%,29.5%
Rural Areas,"1,399","2,585",0.1%,0.1%
Urban Unincorporated Areas,490,"2,265",0.0%,0.1%


In [28]:
pd.options.display.float_format = '{:0,.0f}'.format

equity_dict = {'equity_focus_areas_2023__efa_poc': 'People of Color',
                      'equity_focus_areas_2023__efa_pov200': 'Poverty',
                        'equity_focus_areas_2023__efa_lep': 'LEP',
                      'equity_focus_areas_2023__efa_older': 'Older',
                      'equity_focus_areas_2023__efa_youth': 'Youth',
            'equity_focus_areas_2023__efa_dis': 'Disability'}

df = hct_hh_df[hct_hh_df['group'].isin(equity_dict.values())].copy()
df['EFA Type'] = df['index'].map({
                                0: 'Below Regional Average', 
                                1: 'Above Regional Average', 
                                2: 'Higher Share of Equity Population',
                                })

df = df.pivot_table(index=['group','EFA Type'], columns='distance', values='emptot_p', aggfunc='sum').reset_index()

# Merge with total households to get shares of group
efa_pop_tot = df_lu[equity_dict.keys()].sum().to_dict()

df[0.25] = df[0.25].astype("float")
df[0.50] = df[0.50].astype("float")

for lu_col, label in equity_dict.items():
    df.loc[df['group'] == label, "Total Jobs"] = efa_pop_tot[lu_col]
df['% of Households (1/4 Mile)'] = df[0.25]/df['Total Jobs']
df['% of Households (1/2 Mile)'] = df[0.5]/df['Total Jobs']


df.rename(columns={'group': 'Group', 0.25: '1/4 Mile', 0.5: '1/2 Mile'}, inplace=True)
df = df.rename_axis(None, axis=1)
df.drop('Total Jobs', inplace=True, axis=1)
output = df.to_html(formatters={
    '% of Jobs (1/4 Mile)': '{:0,.1%}'.format,
    '% of Jobs (1/2 Mile)': '{:0,.1%}'.format,
}, index=False)
display(HTML(output))

Group,EFA Type,1/4 Mile,1/2 Mile,% of Households (1/4 Mile),% of Households (1/2 Mile)
Disability,Above Regional Average,"134,999","194,613",0,0
Disability,Below Regional Average,"274,778","466,829",0,1
Disability,Higher Share of Equity Population,"193,671","251,796",0,0
LEP,Above Regional Average,"172,501","278,633",0,0
LEP,Below Regional Average,"270,418","398,489",0,1
LEP,Higher Share of Equity Population,"160,529","236,116",0,0
Older,Above Regional Average,"154,578","216,339",0,0
Older,Below Regional Average,"390,490","598,924",0,1
Older,Higher Share of Equity Population,"58,380","97,975",0,0
People of Color,Above Regional Average,"323,219","421,251",1,1


# Transit Access Mode
Percent of Transit Trips Accessed by Walking

In [29]:
pd.options.display.float_format = '{:0,.1%}'.format
df = pd.read_csv(r'../../../../outputs/agg/dash/trip_mode_by_tour_mode.csv')
_df = df.copy()
pnr_transit_trips = _df[(_df['tmodetp'] == 'Park') & (_df['mode'] == 'Transit')]['trexpfac'].sum()
walk_transit_trips = _df[(_df['tmodetp'] == 'Transit') & (_df['mode'] == 'Transit')]['trexpfac'].sum()
reg_access = (walk_transit_trips/(walk_transit_trips+pnr_transit_trips))
pd.DataFrame([reg_access], columns=['% Transit trips Accessed by Walking'], index=['Region'])


,% Transit trips Accessed by Walking
Region,95.1%


In [30]:
pd.options.display.float_format = '{:0,.1%}'.format
_df = df.copy()
results_df = pd.DataFrame()
list_50 = ['hh_racial','hh_poverty']
# for col in ['hh_disability','hh_elderly','hh_english','hh_poverty','hh_racial','hh_youth']:
for col in summary_config["hh_equity_geogs"]:
    pnr_transit_trips = _df[(_df[col] == 1) & 
                            (_df['tmodetp'] == 'Park') &
                            (_df['mode'] == 'Transit')]['trexpfac'].sum()
    walk_transit_trips = _df[(_df[col] == 1) & 
                                (_df['tmodetp'] == 'Transit') & 
                                (_df['mode'] == 'Transit')]['trexpfac'].sum()
    if (pnr_transit_trips > 0) & (walk_transit_trips > 0):
        results_df.loc[col,'% Transit Trips Accessed by Walking'] = walk_transit_trips/(walk_transit_trips+pnr_transit_trips)
# results_df.rename(columns={'_reg':'> Regional Average', '_50': '> 50%'}, inplace=True)
results_df

,% Transit Trips Accessed by Walking
hh_efa_dis,95.7%
hh_efa_older,95.1%
hh_efa_lep,95.7%
hh_efa_pov200,96.0%
hh_efa_poc,96.2%
hh_efa_youth,93.3%


# Transit Equity

In [31]:


myh5 = h5py.File(r'..\..\..\..\inputs\scenario\landuse\hh_and_persons.h5', 'r')
person = pd.read_csv(r'..\..\..\..\outputs\daysim\_person.tsv', sep='\t')
df_trip = pd.read_csv(r'..\..\..\..\outputs\daysim\_trip.tsv', sep='\t')
df_hh = pd.read_csv(r'..\..\..\..\outputs\daysim\_household.tsv', sep='\t')

In [32]:
# Create a DataFrame from the h5 file
df_person = pd.DataFrame()
for col in ['hhno','pno','prace']:
    df_person[col] = myh5['Person'][col][:]

df_trip = df_trip.merge(df_person[['hhno','pno','prace']], on=['hhno','pno'], how='left')

In [33]:
# Create group of work, school, and all other purposes
df_trip['Purpose Type'] = 'Other'
df_trip.loc[df_trip['dpurp']==0, 'Purpose Type'] = 'Home'
df_trip.loc[df_trip['dpurp']==1, 'Purpose Type'] = 'Work'
df_trip.loc[df_trip['dpurp']==2, 'Purpose Type'] = 'School'

# Rename path types
df_trip.loc[df_trip['pathtype']==1, 'Path Type'] = 'Drive'
df_trip.loc[df_trip['pathtype']==3, 'Path Type'] = 'Bus'
df_trip.loc[df_trip['pathtype']==4, 'Path Type'] = 'Light Rail'
df_trip.loc[df_trip['pathtype']==5, 'Path Type'] = 'Ferry 1'
df_trip.loc[df_trip['pathtype']==6, 'Path Type'] = 'Commuter Rail'
df_trip.loc[df_trip['pathtype']==7, 'Path Type'] = 'Ferry 2'

df = pd.pivot_table(df_trip[(df_trip['dpurp']!=0) & (~df_trip['pathtype'].isin([0,'Drive']))], values='trexpfac', index='Path Type', columns='Purpose Type', aggfunc='sum')

df['Total'] = df.sum(axis=1)
for row in df.index:
    for col in ['Work','School','Other']:
        df.loc[row, col + ' %'] = df.loc[row, col] / df.loc[row, 'Total']

df[['Work %','School %','Other %']] = df[['Work %','School %','Other %']].applymap("{:,.1%}".format)
df[['Work %','School %','Other %']]

C:\Users\Modeller\AppData\Local\Temp\ipykernel_44072\2330724568.py:22: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df[['Work %','School %','Other %']] = df[['Work %','School %','Other %']].applymap("{:,.1%}".format)


Purpose Type,Work %,School %,Other %
Path Type,,,
Bus,16.1%,14.4%,69.5%
Commuter Rail,52.0%,1.8%,46.2%
Drive,20.2%,6.2%,73.6%
Ferry 1,41.6%,15.2%,43.1%
Ferry 2,61.5%,1.9%,36.6%
Light Rail,31.9%,14.2%,53.9%


In [34]:

race_dict = {
    '1': 'White alone',
    '2': 'Black or African American alone',
    '3': 'American Indian alone',
    '4': 'Alaska Native alone',
    '5': 'American Indian and Alaska Native tribes specified',
    '6': 'Asian alone',
    '7': 'Native Hawaiian and Other Pacific Islander alone',
    '8': 'Some Other Race alone',
    '9': 'Two or More Races'
}


df_trip['Race'] = df_trip['prace'].astype('int').astype('str').map(race_dict)

In [35]:
# Submode Ridership by race
df = pd.pivot_table(df_trip[(df_trip['pathtype']!="Drive")], values='trexpfac', index='Path Type', columns='Race', aggfunc="sum")
race_cols = ['White alone','Black or African American alone','American Indian alone',
             'Alaska Native alone','American Indian and Alaska Native tribes specified','Asian alone',
             'Native Hawaiian and Other Pacific Islander alone','Some Other Race alone','Two or More Races']
# Calculate shares of total by Race for each Path Type
df['Total'] = df.sum(axis=1)
df = df.astype('float')
for row in df.index:
    for col in race_cols:
        df.loc[row, col] = df.loc[row, col] / df.loc[row, 'Total']
df[race_cols] = df[race_cols].map("{:,.1%}".format)
df[race_cols]

Race,White alone,Black or African American alone,American Indian alone,Alaska Native alone,American Indian and Alaska Native tribes specified,Asian alone,Native Hawaiian and Other Pacific Islander alone,Some Other Race alone,Two or More Races
Path Type,,,,,,,,,
Bus,58.8%,7.8%,0.6%,0.1%,0.1%,18.8%,0.7%,4.0%,9.1%
Commuter Rail,62.6%,7.4%,0.8%,0.1%,0.1%,11.9%,2.3%,5.0%,9.8%
Drive,64.6%,5.6%,0.6%,0.1%,0.1%,14.8%,1.0%,4.3%,9.0%
Ferry 1,77.5%,2.0%,0.5%,0.1%,0.3%,5.7%,2.1%,2.6%,9.2%
Ferry 2,82.0%,1.9%,0.9%,0.1%,0.2%,4.4%,0.9%,2.6%,7.0%
Light Rail,57.1%,8.7%,0.6%,0.1%,0.1%,20.7%,0.6%,3.6%,8.4%


In [36]:
df_trip = df_trip.merge(df_hh[['hhno','hhincome']], on=['hhno'], how='left')

In [37]:
# Create bins for hhincome
bins = [0, 20000, 40000, 60000, 80000, 100000, 120000, 140000, 160000, 180000, 200000, np.inf]
labels = ['<20k', '20-40k', '40-60k', '60-80k', '80-100k', '100-120k', '120-140k', '140-160k', '160-180k', '180-200k', '>200k']
df_trip['Income Bin'] = pd.cut(df_trip['hhincome'], bins=bins, labels=labels, right=False)
df_trip['Income Bin'] = df_trip['Income Bin'].astype('str')

# Transit Submodes by Income
df = pd.pivot_table(df_trip[(df_trip['Path Type']!="Drive")], values='trexpfac', index='Path Type', columns='Income Bin', aggfunc="sum")

# display labels with no significant figs
df[labels] = df[labels].map("{:0,.0f}".format)
df[labels]

Income Bin,<20k,20-40k,40-60k,60-80k,80-100k,100-120k,120-140k,140-160k,160-180k,180-200k,>200k
Path Type,,,,,,,,,,,
Bus,"35,171","27,776","23,230","20,390","19,091","14,943","14,499","13,522","11,040","9,905","60,626"
Commuter Rail,179,336,621,646,960,748,736,587,479,510,"1,733"
Ferry 1,210,243,353,397,324,348,288,228,222,174,725
Ferry 2,124,311,472,774,860,944,786,708,555,443,"2,664"
Light Rail,"9,378","7,440","7,018","6,858","6,413","5,327","5,139","4,789","3,947","3,193","24,151"


In [38]:
pd.pivot_table(
    df_trip[df_trip['Path Type'] != "Drive"],
    values='hhincome',
    index='Path Type',
    aggfunc="median"
).astype(int).map(lambda x: "${:,.0f}".format(x))

,hhincome
Path Type,
Bus,"$99,537"
Commuter Rail,"$127,518"
Ferry 1,"$113,419"
Ferry 2,"$141,962"
Light Rail,"$117,951"
